# Random Generator

## Notebook Description

This Notebook shall randomly generate Jobs and Machines.<br>

It randomly sets the environmental hyperparameters 
- max_runtime
- max_init_runtime

of the maximum permitted processing time of any Job on any Machine and the maximum permitted initial machine occupation time.<br>

It then creates the Jobs and Machiines. The respective processing times, deadlines, weights and initial occupation times are uniformly sampled over the range from 0 or 1 until the maximum value defined by the respective environmental hyperparameter.

## Code

In [ ]:
import random
import pickle
import os
import import_ipynb
from Jobs_and_Machines import *
from Global_Variables import *

### Create File of Hyperparameters

For the case that several Job Scheduling Problems shall be created over the same set of hyperparameters.

In [ ]:
#read hyperparameters of an already existing file
def read_max_parameters(ind, prnt=False):
    
    parameters = []
    
    #set numbers consist of 4 digits for better organization in directory, so file 1 is file 0001
    ind_str = "0"*(2-len(str(ind))) + str(ind)
    
    #open existing file of hyperparameters
    with open(f'MaxValuesSets/MaxValues_{ind_str}/MaxValues_{ind_str}.txt', 'r') as f:
        for line in f:
            #every line is a hyperparameter
            parameters.append(int(line.split(" = ")[1]))
            #if desired, print the hyperparameters
            if prnt:
                print(line.replace("\n",""))
        
    return(parameters)

In [ ]:
#we will define a set of maximal values/hyperparameters and save it as a file to create a list of jobs and machines later on
def generate_max_values_sets(ind):
    
    #add zeros in front of index of file to store files more organizedly until 4 digits. File 1 becomes file 0001. 
    ind_str = "0"*(2-len(str(ind))) + str(ind)
    
    #check if required directories to save file containing hyperparameters exist already
    if not os.path.exists('MaxValuesSets'):
        os.mkdir('MaxValuesSets') #head directory to save all data related files
    if not os.path.exists(f'MaxValuesSets/MaxValues_{ind_str}'):
        os.mkdir(f'MaxValuesSets/MaxValues_{ind_str}') #one directory for every max values set environment
    if not os.path.exists(f'MaxValuesSets/MaxValues_{ind_str}/Jobs_and_Machines_{ind_str}'): 
        os.mkdir(f'MaxValuesSets/MaxValues_{ind_str}/Jobs_and_Machines_{ind_str}') #directory for list of jobs and machines
    if not os.path.exists(f'MaxValuesSets/MaxValues_{ind_str}/States_{ind_str}'):
        os.mkdir(f'MaxValuesSets/MaxValues_{ind_str}/States_{ind_str}') #directory for all states
    if not os.path.exists(f'MaxValuesSets/MaxValues_{ind_str}/Data_{ind_str}'):
        os.mkdir(f'MaxValuesSets/MaxValues_{ind_str}/Data_{ind_str}') #directory for NN data created out of states
    
    #we check if max values set already exists
    if os.path.exists(f'MaxValuesSets/MaxValues_{ind_str}/MaxValues_{ind_str}.txt'):
        #check if they match, else raise an error
        if [n,m,max_init_runtime,max_runtime,max_deadline,max_weight] != read_max_parameters(ind):
            raise ValueError(f'Max Values Set {ind_str} already exists but does not match the given parameters!')
    #otherwise, save all parameters 
    else:
        text_to_save = f"n = {n}\
        \nm = {m}\
        \nmax_init_runtime = {max_init_runtime}\
        \nmax_runtime = {max_runtime}\
        \nmax_deadline = {max_deadline}\
        \nmax_weight = {max_weight}"
        #create .txt-file for parameters
        with open(f'MaxValuesSets/MaxValues_{ind_str}/MaxValues_{ind_str}.txt', 'w') as f:
            f.write(text_to_save)

### Generate Random Jobs and Machines

In [ ]:
#generate random initial machine occupation times
def generate_random_init_runtimes(m, max_init_runtime):
    
    #generate random initial runtimes up to definied maximal vlaue
    rand_init_runtimes = [random.randint(0,max_init_runtime) for i in range(m)]
    #set one to zero to create the starting point for the Job Scheduling Problem
    if 0 not in rand_init_runtimes:
        rand_init_runtimes[random.randrange(m)] = 0
        
    return rand_init_runtimes

In [ ]:
#generate list of random machines
def generate_random_machines(m, max_init_runtime, max_deadline, max_weight, prnt=False):
    
    #sample initial machine occupation times
    rand_init_runtimes = generate_random_init_runtimes(m, max_init_runtime)
    
    #generate machines with random deadlines, weights and initial occupation times up to the defined maximal value
    random_list_machines = [machines(rand_init_runtimes[i],
                                     random.randint(0,max_deadline), 
                                     random.randint(1,max_weight))
                            for i in range(m)]
    
    #appropietly prepare machines
    if prnt:
        #better readability
        print("")
    #sort them accordingly
    random_list_machines = prep_machines(random_list_machines, prnt=prnt)
    
    return random_list_machines

In [ ]:
#generate list of random jobs
def generate_random_jobs(n, m, max_runtime, max_deadline, max_weight, prnt=False):
    
    #generate jobs with random processing times, deadlines and weights up to the defined maximal value (processing time > 0)
    random_list_jobs = [jobs([random.randint(1,max_runtime) for i in range(m)], 
                             random.randint(0,max_deadline), 
                             random.randint(1,max_weight)) 
                        for j in range(n)]
    
    #appropietly prepare jobs
    if prnt:
        #better readability
        print("")
    #sort them accordingly
    random_list_jobs = prep_jobs(random_list_jobs, prnt=prnt)
    
    return random_list_jobs

The following function generates a list of random Jobs and a list of random Machines.<br>
It can be run with a saved file of hyperparameters or with a temporarily generated set of them within the subsequent function "generate_random_environment".

In [ ]:
#generate a random Job Scheduling Problem by randomly generating the jobs and machines
def generate_random_JS(MVS, JS, prnt=False):
    
    #generate list of random machines with random initial runtimes
    random_list_machines = generate_random_machines(m, max_init_runtime, max_deadline, max_weight, prnt=prnt)
    #generate list of random jobs
    random_list_jobs = generate_random_jobs (n, m, max_runtime, max_deadline, max_weight, prnt=prnt)
    
    #in case that a saved file of hyperparameters was used, also save the jobs and machines in a correspondingly indexed file
    if type(MVS) == int and type(JS) == int:
        #save the generated lists
        MVS_str = "0"*(2-len(str(MVS))) + str(MVS)
        JS_str = "0"*(4-len(str(JS))) + str(JS)
        #path is indexed by the used file of hyperparameters
        path = f'MaxValuesSets/MaxValues_{MVS_str}/Jobs_and_Machines_{MVS_str}/random_lists_{MVS_str}_{JS_str}.pickle'
        with open(path, 'wb') as f:
            pickle.dump((random_list_jobs, 
                         random_list_machines), 
                        f, pickle.HIGHEST_PROTOCOL)

    return random_list_jobs, random_list_machines

### Generate random Hyperparameters, Jobs and Machines for single use

The subsequent function uses the above to generate a random list of Jobs and Machines without using a file of saved hyperparameters, hence sampling temporary hyperparameters for every Job Scheduling Problem.

In [ ]:
#randomly generate the maximum processing time and maximum initial machine occupation time
def generate_random_environment():
    
    #set hyperparameters to global
    global max_runtime, max_init_runtime
    #sample processing time from the range of 33% up to 150% of the maximum deadline
    max_runtime = random.randint(round(max_deadline/3),round(max_deadline*1.5))
    #sample maximum initial machine occupation from the range of 33% up to 100% of the maximum processing time
    max_init_runtime = random.randint(round(max_runtime/3), max_runtime)
    
    #use respective hyperparemeters and the above function to create lists of random jobs and machines
    random_list_jobs, random_list_machines = generate_random_JS("","")
    
    return [max_runtime, max_init_runtime, random_list_jobs, random_list_machines]